In [ ]:
import sys
sys.path.append("..")

import random
import math
from pathlib import Path
from typing import Optional, Callable, List, Tuple, Iterable

import PIL.Image
from PIL import ImageFont, ImageDraw
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as VT
import torchvision.transforms.functional as VF
from torchvision.utils import make_grid

from src.datasets import *
from src.util.image import * 

In [ ]:
torch.cat([torch.zeros(3, 3, 32, 32).unsqueeze(0)] * 2).mean(0).shape

In [ ]:
shape = (1, 32, 32)
total = 10_000_000

In [ ]:
#help(ImageDraw.ImageDraw.text)
help(ImageFont.ImageFont)

In [ ]:
font = ImageFont.truetype("/usr/share/vlc/skins2/fonts/FreeSans.ttf")
font.getname()
#img = font.getmask("a")
#img#PIL.Image.frombuffer("L", (6, 5), img)


In [ ]:
REAL_FONTS = [
    "/usr/share/fonts/truetype/fonts-japanese-mincho.ttf",
    "/usr/share/fonts/truetype/fonts-japanese-gothic.ttf",
    "/usr/share/fonts/truetype/ubuntu/UbuntuMono-BI.ttf",
    "/usr/share/fonts/truetype/ubuntu/Ubuntu-RI.ttf",
    "/usr/share/fonts/truetype/ubuntu/UbuntuMono-R.ttf",
    "/usr/share/fonts/truetype/ubuntu/Ubuntu-BI.ttf",
    "/usr/share/fonts/truetype/ubuntu/Ubuntu-LI.ttf",
    "/usr/share/fonts/truetype/ubuntu/Ubuntu-MI.ttf",
    "/usr/share/fonts/truetype/ubuntu/Ubuntu-L.ttf",
    "/usr/share/fonts/truetype/ubuntu/Ubuntu-B.ttf",
    "/usr/share/fonts/truetype/ubuntu/Ubuntu-C.ttf",
    "/usr/share/fonts/truetype/ubuntu/UbuntuMono-RI.ttf",
    "/usr/share/fonts/truetype/ubuntu/Ubuntu-M.ttf",
    "/usr/share/fonts/truetype/ubuntu/Ubuntu-R.ttf",
    "/usr/share/fonts/truetype/ubuntu/UbuntuMono-B.ttf",
    "/usr/share/fonts/truetype/roboto/hinted/Roboto-Black.ttf",
    "/usr/share/fonts/truetype/roboto/hinted/Roboto-ThinItalic.ttf",
    "/usr/share/fonts/truetype/roboto/hinted/Roboto-Thin.ttf",
    "/usr/share/fonts/truetype/roboto/hinted/Roboto-Italic.ttf",
    "/usr/share/fonts/truetype/roboto/hinted/Roboto-BlackItalic.ttf",
    "/usr/share/fonts/truetype/roboto/hinted/RobotoCondensed-Light.ttf",
    "/usr/share/fonts/truetype/roboto/hinted/RobotoCondensed-Bold.ttf",
    "/usr/share/fonts/truetype/roboto/hinted/Roboto-Medium.ttf",
    "/usr/share/fonts/truetype/roboto/hinted/Roboto-Bold.ttf",
    "/usr/share/fonts/truetype/roboto/hinted/RobotoCondensed-LightItalic.ttf",
    "/usr/share/fonts/truetype/roboto/hinted/RobotoCondensed-Italic.ttf",
    "/usr/share/fonts/truetype/roboto/hinted/Roboto-Regular.ttf",
    "/usr/share/fonts/truetype/roboto/hinted/RobotoCondensed-Regular.ttf",
    "/usr/share/fonts/truetype/roboto/hinted/Roboto-Light.ttf",
    "/usr/share/fonts/truetype/roboto/hinted/Roboto-BoldItalic.ttf",
    "/usr/share/fonts/truetype/roboto/hinted/RobotoCondensed-BoldItalic.ttf",
    "/usr/share/fonts/truetype/roboto/hinted/Roboto-LightItalic.ttf",
    "/usr/share/fonts/truetype/roboto/hinted/Roboto-MediumItalic.ttf",
    "/usr/share/fonts/truetype/junicode/Junicode-BoldItalic.ttf",
    "/usr/share/fonts/truetype/junicode/Junicode-Italic.ttf",
    "/usr/share/fonts/truetype/junicode/Junicode-Bold.ttf",
    "/usr/share/fonts/truetype/junicode/Junicode.ttf",
    "/usr/share/fonts/truetype/tibetan-machine/TibetanMachineUni.ttf",
    "/usr/share/fonts/truetype/malayalam/Dyuthi.ttf",
    "/usr/share/fonts/truetype/malayalam/Suruma.ttf",
    "/usr/share/fonts/truetype/malayalam/AnjaliOldLipi.ttf",
    "/usr/share/fonts/truetype/malayalam/Meera.ttf",
    "/usr/share/fonts/truetype/malayalam/Rachana-Regular.ttf",
    "/usr/share/fonts/truetype/malayalam/Karumbi.ttf",
    "/usr/share/fonts/truetype/malayalam/Uroob.ttf",
    "/usr/share/fonts/truetype/malayalam/Chilanka-Regular.ttf",
    "/usr/share/fonts/truetype/malayalam/Keraleeyam.ttf",
    "/usr/share/fonts/truetype/malayalam/Rachana-Bold.ttf",
    "/usr/share/fonts/truetype/gentium/GentiumAlt-R.ttf",
    "/usr/share/fonts/truetype/gentium/Gentium-I.ttf",
    "/usr/share/fonts/truetype/gentium/Gentium-R.ttf",
    "/usr/share/fonts/truetype/gentium/GentiumAlt-I.ttf",
    "/usr/share/fonts/truetype/Sarai/Sarai.ttf",
    "/usr/share/fonts/truetype/comfortaa/Comfortaa-Light.ttf",
    "/usr/share/fonts/truetype/comfortaa/Comfortaa-Bold.ttf",
    "/usr/share/fonts/truetype/comfortaa/Comfortaa-Regular.ttf",
    "/usr/share/fonts/truetype/lao/Phetsarath_OT.ttf",
    "/usr/share/fonts/truetype/fonts-beng-extra/mitra.ttf",
    "/usr/share/fonts/truetype/fonts-beng-extra/LikhanNormal.ttf",
    "/usr/share/fonts/truetype/fonts-beng-extra/MuktiNarrowBold.ttf",
    "/usr/share/fonts/truetype/fonts-beng-extra/ani.ttf",
    "/usr/share/fonts/truetype/fonts-beng-extra/JamrulNormal.ttf",
    "/usr/share/fonts/truetype/fonts-beng-extra/MuktiNarrow.ttf",
    "/usr/share/fonts/truetype/lohit-devanagari/Lohit-Devanagari.ttf",
    "/usr/share/fonts/truetype/ebgaramond/EBGaramond12-Italic.ttf",
    "/usr/share/fonts/truetype/ebgaramond/EBGaramond12-Regular.ttf",
    "/usr/share/fonts/truetype/ebgaramond/EBGaramondSC08-Regular.ttf",
    "/usr/share/fonts/truetype/ebgaramond/EBGaramond08-Regular.ttf",
    "/usr/share/fonts/truetype/ebgaramond/EBGaramondSC12-Regular.ttf",
    "/usr/share/fonts/truetype/ebgaramond/EBGaramond12-AllSC.ttf",
    "/usr/share/fonts/truetype/ebgaramond/EBGaramond08-Italic.ttf",
    "/usr/share/fonts/truetype/padauk/Padauk-Bold.ttf",
    "/usr/share/fonts/truetype/padauk/PadaukBook-Regular.ttf",
    "/usr/share/fonts/truetype/padauk/Padauk-Regular.ttf",
    "/usr/share/fonts/truetype/padauk/PadaukBook-Bold.ttf",
    "/usr/share/fonts/truetype/lato/Lato-Medium.ttf",
    "/usr/share/fonts/truetype/lato/Lato-MediumItalic.ttf",
    "/usr/share/fonts/truetype/lato/Lato-Thin.ttf",
    "/usr/share/fonts/truetype/lato/Lato-Heavy.ttf",
    "/usr/share/fonts/truetype/lato/Lato-HeavyItalic.ttf",
    "/usr/share/fonts/truetype/lato/Lato-Light.ttf",
    "/usr/share/fonts/truetype/lato/Lato-Regular.ttf",
    "/usr/share/fonts/truetype/lato/Lato-BoldItalic.ttf",
    "/usr/share/fonts/truetype/lato/Lato-Hairline.ttf",
    "/usr/share/fonts/truetype/lato/Lato-Italic.ttf",
    "/usr/share/fonts/truetype/lato/Lato-ThinItalic.ttf",
    "/usr/share/fonts/truetype/lato/Lato-LightItalic.ttf",
    "/usr/share/fonts/truetype/lato/Lato-Semibold.ttf",
    "/usr/share/fonts/truetype/lato/Lato-HairlineItalic.ttf",
    "/usr/share/fonts/truetype/lato/Lato-SemiboldItalic.ttf",
    "/usr/share/fonts/truetype/lato/Lato-BlackItalic.ttf",
    "/usr/share/fonts/truetype/lato/Lato-Black.ttf",
    "/usr/share/fonts/truetype/lato/Lato-Bold.ttf",
    "/usr/share/fonts/truetype/croscore/Arimo-Italic.ttf",
    "/usr/share/fonts/truetype/croscore/Arimo-BoldItalic.ttf",
    "/usr/share/fonts/truetype/croscore/Cousine-Italic.ttf",
    "/usr/share/fonts/truetype/croscore/Cousine-BoldItalic.ttf",
    "/usr/share/fonts/truetype/croscore/Arimo-Regular.ttf",
    "/usr/share/fonts/truetype/croscore/Tinos-Regular.ttf",
    "/usr/share/fonts/truetype/croscore/Cousine-Bold.ttf",
    "/usr/share/fonts/truetype/croscore/Tinos-Bold.ttf",
    "/usr/share/fonts/truetype/croscore/Arimo-Bold.ttf",
    "/usr/share/fonts/truetype/croscore/Tinos-Italic.ttf",
    "/usr/share/fonts/truetype/croscore/Cousine-Regular.ttf",
    "/usr/share/fonts/truetype/croscore/Tinos-BoldItalic.ttf",
    "/usr/share/fonts/truetype/noto/NotoSerif-Bold.ttf",
    "/usr/share/fonts/truetype/noto/NotoSansDisplay-Bold.ttf",
    "/usr/share/fonts/truetype/noto/NotoSerif-BoldItalic.ttf",
    "/usr/share/fonts/truetype/noto/NotoSerif-Italic.ttf",
    "/usr/share/fonts/truetype/noto/NotoSansMono-Bold.ttf",
    "/usr/share/fonts/truetype/noto/NotoSans-Regular.ttf",
    "/usr/share/fonts/truetype/noto/NotoSerifDisplay-BoldItalic.ttf",
    "/usr/share/fonts/truetype/noto/NotoSansDisplay-BoldItalic.ttf",
    "/usr/share/fonts/truetype/noto/NotoSans-Bold.ttf",
    "/usr/share/fonts/truetype/liberation2/LiberationMono-Italic.ttf",
    "/usr/share/fonts/truetype/liberation2/LiberationSans-Bold.ttf",
    "/usr/share/fonts/truetype/liberation2/LiberationMono-Bold.ttf",
    "/usr/share/fonts/truetype/liberation2/LiberationSerif-BoldItalic.ttf",
    "/usr/share/fonts/truetype/liberation2/LiberationSans-Regular.ttf",
    "/usr/share/fonts/truetype/liberation2/LiberationSans-Italic.ttf",
    "/usr/share/fonts/truetype/liberation2/LiberationSans-BoldItalic.ttf",
    "/usr/share/fonts/truetype/liberation2/LiberationSerif-Italic.ttf",
    "/usr/share/fonts/truetype/liberation2/LiberationSerif-Regular.ttf",
    "/usr/share/fonts/truetype/liberation2/LiberationMono-Regular.ttf",
    "/usr/share/fonts/truetype/liberation2/LiberationSerif-Bold.ttf",
    "/usr/share/fonts/truetype/liberation2/LiberationMono-BoldItalic.ttf",
    "/usr/share/fonts/truetype/gentium-basic/GenBasBI.ttf",
    "/usr/share/fonts/truetype/gentium-basic/GenBkBasI.ttf",
    "/usr/share/fonts/truetype/gentium-basic/GenBkBasR.ttf",
    "/usr/share/fonts/truetype/gentium-basic/GenBasI.ttf",
    "/usr/share/fonts/truetype/gentium-basic/GenBasB.ttf",
    "/usr/share/fonts/truetype/gentium-basic/GenBkBasB.ttf",
    "/usr/share/fonts/truetype/gentium-basic/GenBkBasBI.ttf",
    "/usr/share/fonts/truetype/gentium-basic/GenBasR.ttf",
    "/usr/share/fonts/truetype/liberation/LiberationMono-Italic.ttf",
    "/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf",
    "/usr/share/fonts/truetype/liberation/LiberationMono-Bold.ttf",
    "/usr/share/fonts/truetype/liberation/LiberationSansNarrow-BoldItalic.ttf",
    "/usr/share/fonts/truetype/liberation/LiberationSerif-BoldItalic.ttf",
    "/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf",
    "/usr/share/fonts/truetype/liberation/LiberationSans-Italic.ttf",
    "/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Bold.ttf",
    "/usr/share/fonts/truetype/liberation/LiberationSans-BoldItalic.ttf",
    "/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
    "/usr/share/fonts/truetype/liberation/LiberationSerif-Italic.ttf",
    "/usr/share/fonts/truetype/liberation/LiberationSerif-Regular.ttf",
    "/usr/share/fonts/truetype/liberation/LiberationMono-Regular.ttf",
    "/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Italic.ttf",
    "/usr/share/fonts/truetype/liberation/LiberationSerif-Bold.ttf",
    "/usr/share/fonts/truetype/liberation/LiberationMono-BoldItalic.ttf",
    "/usr/share/fonts/truetype/Gargi/Gargi.ttf",
    "/usr/share/fonts/truetype/abyssinica/AbyssinicaSIL-R.ttf",
    "/usr/share/fonts/truetype/tlwg/Kinnari-BoldOblique.ttf",
    "/usr/share/fonts/truetype/tlwg/Waree-Bold.ttf",
    "/usr/share/fonts/truetype/tlwg/TlwgTypo-BoldOblique.ttf",
    "/usr/share/fonts/truetype/tlwg/Laksaman-Bold.ttf",
    "/usr/share/fonts/truetype/tlwg/Garuda-Bold.ttf",
    "/usr/share/fonts/truetype/tlwg/Purisa-BoldOblique.ttf",
    "/usr/share/fonts/truetype/tlwg/TlwgMono.ttf",
    "/usr/share/fonts/truetype/tlwg/Loma.ttf",
    "/usr/share/fonts/truetype/tlwg/TlwgTypewriter-Oblique.ttf",
    "/usr/share/fonts/truetype/tlwg/Norasi-Bold.ttf",
    "/usr/share/fonts/truetype/tlwg/Umpush-Bold.ttf",
    "/usr/share/fonts/truetype/tlwg/Umpush-Oblique.ttf",
    "/usr/share/fonts/truetype/tlwg/TlwgMono-Bold.ttf",
    "/usr/share/fonts/truetype/tlwg/Norasi-Oblique.ttf",
    "/usr/share/fonts/truetype/tlwg/Loma-BoldOblique.ttf",
    "/usr/share/fonts/truetype/tlwg/Garuda-Oblique.ttf",
    "/usr/share/fonts/truetype/tlwg/Norasi.ttf",
    "/usr/share/fonts/truetype/tlwg/TlwgTypist-BoldOblique.ttf",
    "/usr/share/fonts/truetype/tlwg/TlwgTypo-Bold.ttf",
    "/usr/share/fonts/truetype/tlwg/Norasi-BoldOblique.ttf",
    "/usr/share/fonts/truetype/tlwg/Kinnari-BoldItalic.ttf",
    "/usr/share/fonts/truetype/tlwg/Umpush-LightOblique.ttf",
    "/usr/share/fonts/truetype/tlwg/Umpush-BoldOblique.ttf",
    "/usr/share/fonts/truetype/tlwg/Sawasdee-Oblique.ttf",
    "/usr/share/fonts/truetype/tlwg/Umpush.ttf",
    "/usr/share/fonts/truetype/tlwg/TlwgTypewriter-BoldOblique.ttf",
    "/usr/share/fonts/truetype/tlwg/Waree-Oblique.ttf",
    "/usr/share/fonts/truetype/tlwg/Laksaman-Italic.ttf",
    "/usr/share/fonts/truetype/tlwg/Loma-Bold.ttf",
    "/usr/share/fonts/truetype/tlwg/Laksaman-BoldItalic.ttf",
    "/usr/share/fonts/truetype/tlwg/Sawasdee-BoldOblique.ttf",
    "/usr/share/fonts/truetype/tlwg/TlwgTypist-Bold.ttf",
    "/usr/share/fonts/truetype/tlwg/Sawasdee-Bold.ttf",
    "/usr/share/fonts/truetype/tlwg/Kinnari-Bold.ttf",
    "/usr/share/fonts/truetype/tlwg/Kinnari-Oblique.ttf",
    "/usr/share/fonts/truetype/tlwg/Purisa-Oblique.ttf",
    "/usr/share/fonts/truetype/tlwg/Garuda-BoldOblique.ttf",
    "/usr/share/fonts/truetype/tlwg/Sawasdee.ttf",
    "/usr/share/fonts/truetype/tlwg/TlwgMono-BoldOblique.ttf",
    "/usr/share/fonts/truetype/tlwg/TlwgTypo-Oblique.ttf",
    "/usr/share/fonts/truetype/tlwg/Norasi-Italic.ttf",
    "/usr/share/fonts/truetype/tlwg/Umpush-Light.ttf",
    "/usr/share/fonts/truetype/tlwg/TlwgMono-Oblique.ttf",
    "/usr/share/fonts/truetype/tlwg/TlwgTypo.ttf",
    "/usr/share/fonts/truetype/tlwg/Garuda.ttf",
    "/usr/share/fonts/truetype/tlwg/Kinnari-Italic.ttf",
    "/usr/share/fonts/truetype/tlwg/TlwgTypist.ttf",
    "/usr/share/fonts/truetype/tlwg/Waree.ttf",
    "/usr/share/fonts/truetype/tlwg/Purisa.ttf",
    "/usr/share/fonts/truetype/tlwg/Waree-BoldOblique.ttf",
    "/usr/share/fonts/truetype/tlwg/Purisa-Bold.ttf",
    "/usr/share/fonts/truetype/tlwg/Kinnari.ttf",
    "/usr/share/fonts/truetype/tlwg/TlwgTypewriter-Bold.ttf",
    "/usr/share/fonts/truetype/tlwg/TlwgTypewriter.ttf",
    "/usr/share/fonts/truetype/tlwg/TlwgTypist-Oblique.ttf",
    "/usr/share/fonts/truetype/tlwg/Laksaman.ttf",
    "/usr/share/fonts/truetype/tlwg/Loma-Oblique.ttf",
    "/usr/share/fonts/truetype/tlwg/Norasi-BoldItalic.ttf",
    "/usr/share/fonts/truetype/ttf-khmeros-core/KhmerOSsys.ttf",
    "/usr/share/fonts/truetype/ttf-khmeros-core/KhmerOS.ttf",
    "/usr/share/fonts/truetype/crosextra/Caladea-Italic.ttf",
    "/usr/share/fonts/truetype/crosextra/Carlito-Italic.ttf",
    "/usr/share/fonts/truetype/crosextra/Caladea-BoldItalic.ttf",
    "/usr/share/fonts/truetype/crosextra/Caladea-Regular.ttf",
    "/usr/share/fonts/truetype/crosextra/Caladea-Bold.ttf",
    "/usr/share/fonts/truetype/crosextra/Carlito-Regular.ttf",
    "/usr/share/fonts/truetype/crosextra/Carlito-Bold.ttf",
    "/usr/share/fonts/truetype/crosextra/Carlito-BoldItalic.ttf",
    "/usr/share/fonts/truetype/open-sans/OpenSans-CondLight.ttf",
    "/usr/share/fonts/truetype/open-sans/OpenSans-Semibold.ttf",
    "/usr/share/fonts/truetype/open-sans/OpenSans-CondLightItalic.ttf",
    "/usr/share/fonts/truetype/open-sans/OpenSans-Bold.ttf",
    "/usr/share/fonts/truetype/open-sans/OpenSans-CondBold.ttf",
    "/usr/share/fonts/truetype/open-sans/OpenSans-ExtraBold.ttf",
    "/usr/share/fonts/truetype/open-sans/OpenSans-ExtraBoldItalic.ttf",
    "/usr/share/fonts/truetype/open-sans/OpenSans-LightItalic.ttf",
    "/usr/share/fonts/truetype/open-sans/OpenSans-Italic.ttf",
    "/usr/share/fonts/truetype/open-sans/OpenSans-SemiboldItalic.ttf",
    "/usr/share/fonts/truetype/open-sans/OpenSans-BoldItalic.ttf",
    "/usr/share/fonts/truetype/open-sans/OpenSans-Regular.ttf",
    "/usr/share/fonts/truetype/open-sans/OpenSans-Light.ttf",
    "/usr/share/fonts/truetype/fonts-deva-extra/kalimati.ttf",
    "/usr/share/fonts/truetype/fonts-deva-extra/chandas1-2.ttf",
    "/usr/share/fonts/truetype/fonts-deva-extra/samanata.ttf",
    "/usr/share/fonts/truetype/Nakula/nakula.ttf",
    "/usr/share/fonts/truetype/dejavu/DejaVuSansMono-Bold.ttf",
    "/usr/share/fonts/truetype/dejavu/DejaVuMathTeXGyre.ttf",
    "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",
    "/usr/share/fonts/truetype/dejavu/DejaVuSerifCondensed-Italic.ttf",
    "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf",
    "/usr/share/fonts/truetype/dejavu/DejaVuSerifCondensed-Bold.ttf",
    "/usr/share/fonts/truetype/dejavu/DejaVuSerifCondensed-BoldItalic.ttf",
    "/usr/share/fonts/truetype/dejavu/DejaVuSansMono.ttf",
    "/usr/share/fonts/truetype/dejavu/DejaVuSerif-Bold.ttf",
    "/usr/share/fonts/truetype/dejavu/DejaVuSans-BoldOblique.ttf",
    "/usr/share/fonts/truetype/dejavu/DejaVuSans-Oblique.ttf",
    "/usr/share/fonts/truetype/dejavu/DejaVuSansMono-Oblique.ttf",
    "/usr/share/fonts/truetype/dejavu/DejaVuSansCondensed-BoldOblique.ttf",
    "/usr/share/fonts/truetype/dejavu/DejaVuSansCondensed-Bold.ttf",
    "/usr/share/fonts/truetype/dejavu/DejaVuSansCondensed.ttf",
    "/usr/share/fonts/truetype/dejavu/DejaVuSerif-BoldItalic.ttf",
    "/usr/share/fonts/truetype/dejavu/DejaVuSansMono-BoldOblique.ttf",
    "/usr/share/fonts/truetype/dejavu/DejaVuSansCondensed-Oblique.ttf",
    "/usr/share/fonts/truetype/dejavu/DejaVuSans-ExtraLight.ttf",
    "/usr/share/fonts/truetype/dejavu/DejaVuSerif.ttf",
    "/usr/share/fonts/truetype/dejavu/DejaVuSerifCondensed.ttf",
    "/usr/share/fonts/truetype/dejavu/DejaVuSerif-Italic.ttf",
    "/usr/share/fonts/truetype/fonts-gujr-extra/Rekha.ttf",
    "/usr/share/fonts/truetype/fonts-gujr-extra/padmaa.ttf",
    "/usr/share/fonts/truetype/fonts-gujr-extra/aakar-medium.ttf",
    "/usr/share/fonts/truetype/fonts-gujr-extra/padmaa-Bold.1.1.ttf",
    "/usr/share/fonts/truetype/lohit-telugu/Lohit-Telugu.ttf",
    "/usr/share/fonts/truetype/freefont/FreeSerifBoldItalic.ttf",
    "/usr/share/fonts/truetype/freefont/FreeSerifBold.ttf",
    "/usr/share/fonts/truetype/freefont/FreeSerif.ttf",
    "/usr/share/fonts/truetype/freefont/FreeSerifItalic.ttf",
    "/usr/share/fonts/truetype/freefont/FreeMonoBold.ttf",
    "/usr/share/fonts/truetype/freefont/FreeSansBold.ttf",
    "/usr/share/fonts/truetype/freefont/FreeSansOblique.ttf",
    "/usr/share/fonts/truetype/freefont/FreeMonoBoldOblique.ttf",
    "/usr/share/fonts/truetype/freefont/FreeSans.ttf",
    "/usr/share/fonts/truetype/freefont/FreeSansBoldOblique.ttf",
    "/usr/share/fonts/truetype/freefont/FreeMono.ttf",
    "/usr/share/fonts/truetype/freefont/FreeMonoOblique.ttf",
    "/usr/share/fonts/truetype/pagul/Pagul.ttf",
]

In [ ]:
font_path = Path("/usr/share/fonts/truetype")
font_names = list(font_path.rglob("**/*.ttf"))
fonts = []
for font_name in font_names:
    try:
        font = ImageFont.truetype(str(font_name), 10)
        fonts.append((font_name, font))
    except OSError:
        pass
big_image = PIL.Image.new("L", (200, len(fonts)*12))
draw = ImageDraw.ImageDraw(big_image)
for y, (font_name, font) in enumerate(fonts):
    draw.text((0, y*12), f"{font_name.name} {font.getname()[0]}", fill=(255,), font=font)
    #print(f"\"{font_name}\",")
big_image

In [ ]:
def check_image(image) -> bool:
    if torch.all(image.reshape(3, -1).std(1) < 0.1):
        return False
    return True

def random_text() -> str:
    return "".join(
        chr(random.randint(32, 100))
        for i in range(random.randint(1, 10))
    )
def iter_random_images():
    font_path = Path("~/.local/share/fonts").expanduser()
    fonts = list(font_path.glob("*ttf"))
    fonts = [ImageFont.truetype(str(f)) for f in fonts[:10]]
    for i in range(total):
        image = PIL.Image.new(
            "RGB", (random.randint(8, shape[-1]), random.randint(8, shape[-2])), 
            #color=(random.randrange(256), random.randrange(256), random.randrange(256)),
        )
        draw = ImageDraw.ImageDraw(image)
        for i in range(random.randint(1, random.randint(1, 3))):
            draw.text(
                (random.randrange(image.width//10+1), random.randrange(image.height)), random_text(),
                font=random.choice(fonts),
                fill=(random.randrange(256), random.randrange(256), random.randrange(256)),
            )
            
        image = VF.to_tensor(image)
        if random.random() < .5:
            image = VT.RandomRotation(30)(image)
        image = VF.resize(image, shape[1:])
        
        if check_image(image):
            yield image

def iter_images():
    fonts = []
    for font_name in REAL_FONTS:
        try:
            font = ImageFont.truetype(str(font_name), shape[-1])
            fonts.append(font)
        except OSError:
            pass
    for char in list(range(ord("A"), ord("z"))):
        for font in fonts:
            image = PIL.Image.new(
                "RGB", 
                shape[-2:],
                #(10, 10),
                #(random.randint(8, shape[-1]), random.randint(8, shape[-2])), 
                #color=(random.randrange(256), random.randrange(256), random.randrange(256)),
            )
            draw = ImageDraw.ImageDraw(image)
            draw.text(
                #(random.randrange(image.width//10+1), random.randrange(image.height)), random_text(),
                (0, 0),
                text=chr(char),
                font=font,
                #fill=(random.randrange(256), random.randrange(256), random.randrange(256)),
                #strokewidth=10,
            )

            image = VF.to_tensor(image)
            for i in range(10):
                if i > 0: #random.random() < .5:
                    image = VT.RandomRotation(30, expand=True)(image)
                    image = VF.resize(image, shape[1:])

                if check_image(image):
                    yield image[:shape[0]]

samples = []
total_preview = 32*32
for i in tqdm(iter_images(), total=total_preview):
    i = i.clamp(0, 1)
    #if check_image(i):
    samples.append(i)
    if len(samples) >= total_preview:
        break
VF.to_pil_image(make_grid(samples, nrow=16))

In [ ]:
def store_dataset(
        images: Iterable,
        dtype=torch.float32,
        image_folder="~/Pictures/__diverse/",
        output_filename="../datasets/fonts-regular-32x32.pt",
        max_megabyte=1_000,
):
    tensor_batch = []
    tensor_size = 0
    last_print_size = 0
    for image in tqdm(images):
        if len(image.shape) < 4:
            image = image.unsqueeze(0)
        tensor_batch.append(image.clamp(0, 1))
        tensor_size += math.prod(image.shape) * 4

        if tensor_size - last_print_size > 1024 * 1024 * 50:
            last_print_size = tensor_size

            print(f"size: {tensor_size:,}")

        if tensor_size >= max_megabyte * 1024 * 1024:
            break

    tensor_batch = torch.cat(tensor_batch)
    torch.save(tensor_batch, output_filename)

store_dataset(iter_images())

In [ ]:
ds = TensorDataset(torch.load("../datasets/fonts-regular-32x32.pt"))
#ds = TensorDataset(torch.load("../datasets/diverse-32x32-std01.pt"))
dl = DataLoader(ds, shuffle=True, batch_size=24*24)
for batch in dl:
    img = VF.to_pil_image(make_grid(batch[0], nrow=24))
    break
img